TF_IDF

In [1]:
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# Charger le fichier de mots vides (stopwords) personnalisés
french_stopwords_path = '../../app/models/french.txt'

# Vérifier si le fichier existe et le lire
if os.path.exists(french_stopwords_path):
    with open(french_stopwords_path, 'r', encoding='utf-8') as file:
        french_stopwords = file.read().splitlines()  # Lire chaque ligne comme un mot séparé
    print(french_stopwords[:10])  # 10 premiers mots 
else:
    print(f"Le fichier french.txt est introuvable dans {french_stopwords_path}")

['a', 'ai', 'aie', 'aient', 'aies', 'ait', 'alors', 'as', 'au', 'aucun']


In [4]:
# Charger les données depuis le CSV
df = pd.read_csv('../../RGBD/table_produits/produits.csv')

In [5]:
# les descriptions des produits
descriptions = df['description'].tolist()
# Initialisation du vectoriseur TF-IDF avec la liste personnalisée de mots vides
vectorizer = TfidfVectorizer(stop_words=french_stopwords)
# Transformation des descriptions en vecteurs TF-IDF
tfidf_matrix = vectorizer.fit_transform(descriptions)

In [6]:
# Fonction pour recommander des produits basés sur une requête utilisateur
def recommend_by_query(query, tfidf_matrix, vectorizer, top_n=5):
    # Transformation de la requête en vecteur TF-IDF
    query_tfidf = vectorizer.transform([query])
    
    # Calcul de la similarité cosinus entre la requête et les descriptions des produits
    cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix).flatten()
    
    # Trier les indices des produits les plus similaires
    similar_indices = cosine_similarities.argsort()[-top_n:][::-1]
    
    # Afficher les produits recommandés
    print(f"Produits recommandés pour la requête '{query}':\n")
    for idx in similar_indices:
        print(f"Produit {idx+1}: {df['description'][idx]} - Similarité: {cosine_similarities[idx]:.4f}")

In [7]:
# Exemple d'appel avec la requête "Saveur menthe"
recommend_by_query("Saveur menthe", tfidf_matrix, vectorizer)

Produits recommandés pour la requête 'Saveur menthe':

Produit 1274: Le Menthe Verte Glacée reprend lessaveurs fraîches d'une menthe chlorophylle, dans une recette signée PULP. Prêt à vaper(PAV), il vous est proposé dans plusieurs taux de nicotine au choix (0, 3, 6 et 12mg/ml), à sélectionner à la commande. Le Menthe Verte Glacée est élaboré à partir d'une base PG/VG de 70/30, et vous est livré dans une fiole de 10ml. Le Menthe Verte Glacée est un e-liquide exclusivement destiné au réservoir de votre cigarette électronique. Il est fabriqué en France par Pulp, à partir d'arômes alimentaires, de propylène glycol et de glycérine végétale. Le e-liquide Menthe Verte Glacée vous est livré dans un flacon de 10ml, équipé d'un bouchon avec sécurité enfant et d'un embout fin facilitant le remplissage de votre réservoir. Qui est PULP ? C'est dans leur laboratoire français, et plus particulièrement parisien que les aromaticiens de la marque PULP conçoivent leurs multiples recettes, et ce, toujours

In [8]:
# Requête de l'utilisateur
requete_utilisateur = "Saveur menthe"
X = vectorizer.fit_transform(df['description'])
# Transformer la requête de l'utilisateur en vecteur TF-IDF
requete_vector = vectorizer.transform([requete_utilisateur])
# Calculer la similarité cosinus entre la requête et les descriptions des produits
similarites = cosine_similarity(requete_vector, X)

In [9]:
# Afficher les produits les plus similaires (triés par similarité décroissante)
df['similarity'] = similarites.flatten()
df_sorted = df.sort_values(by='similarity', ascending=False)

In [10]:
df_sorted[['id_produit', 'description', 'similarity']]

,id_produit,description,similarity
1273,1275,Le Menthe Verte Glacée reprend lessaveurs fraî...,0.346166
1342,1344,Pulp vous propose de vapoter une saveur de boi...,0.326076
1249,1251,"Fabriqué en France par Alfaliquid, le Menthe P...",0.310499
99,100,Si les Français de VDLV n'ont plus besoin d'êt...,0.306293
1263,1265,Le Menthe Glaciale est un e-liquide à booster ...,0.275879
...,...,...,...
701,703,Le Cassis Noir est un e-liquide à booster au f...,0.000000
699,701,Les vendanges ont été fructueuses chez The Fuu...,0.000000
695,697,Le Fraise Kiwi Salt est un e-liquide avec nico...,0.000000
685,687,Le Fraise Grenade est un e-liquide fabriqué en...,0.000000


bm25

In [90]:
# Importer la bibliothèque BM25
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import string

In [91]:
# Fonction de prétraitement (tokenisation et suppression de la ponctuation)
def preprocess(text):
    # Tokenisation et suppression de la ponctuation
    tokens = word_tokenize(text.lower())
    return [token for token in tokens if token not in string.punctuation]

In [92]:
# Appliquer le prétraitement sur toutes les descriptions
preprocessed_descriptions = [preprocess(desc) for desc in df['description']]


In [93]:
# Initialiser le modèle BM25
bm25 = BM25Okapi(preprocessed_descriptions)

In [94]:
# Exemple de requête utilisateur
query = "Saveur cassis lime"
# Prétraiter la requête
preprocessed_query = preprocess(query)

In [ ]:


# Calculer la similarité BM25 pour chaque description
scores = bm25.get_scores(preprocessed_query)
# Afficher les scores de similarité pour chaque description
df['similarity_score'] = scores
# Trier les produits par score de similarité
sorted_df = df.sort_values(by='similarity_score', ascending=False)
# Afficher les 5 produits les plus pertinents
print("Produits les plus pertinents pour la requête :")
print(sorted_df[['description', 'similarity_score']].head())


Produits les plus pertinents pour la requête :
                                           description  similarity_score
101  Besoin d'un bon coup de fouet fruité, frais et...         10.749055
369  Le Bloody Lime est un e-liquideà boosterde la ...         10.261856
276  Le Bloody Lime est un e liquide au formatprêt ...         10.119159
28   Le Frozen Lemon And Lime 100ml est un e-liquid...          9.297028
257  Le Lime Berry 100ml est un e-liquide à booster...          9.160395


In [96]:
# Calculer la similarité BM25 pour chaque description
scores = bm25.get_scores(preprocessed_query)
# Afficher les scores de similarité pour chaque description
df['similarity_score'] = scores
# Trier les produits par score de similarité
sorted_df = df.sort_values(by='similarity_score', ascending=False)
# Afficher les 5 produits les plus pertinents
print("Produits les plus pertinents pour la requête 'Saveur cassis lime' :")
print(sorted_df[['description']].head())


Produits les plus pertinents pour la requête 'Saveur cassis lime' :
                                          description
28  Le Frozen Lemon And Lime 100ml est un e-liquid...
44  Fabriqué en France par Secret's Lab, le e-liqu...
29  Fabriqué en France par Secret's Lab, le e-liqu...
69  Retrouvez dessaveurs de cassis et de mentholda...
78  Retrouvez lessaveurs fruitées de cassisdans ce...


In [17]:
# Si vous voulez afficher le produit le plus pertinent :
most_relevant_product = sorted_df.iloc[0]
print(f"\nLe produit le plus pertinent est :\n{most_relevant_product['description']} avec un score de similarité de {most_relevant_product['similarity_score']:.4f}")


Le produit le plus pertinent est :
Besoin d'un bon coup de fouet fruité, frais et acidulé ? Essayez donc le e liquide Grapefruit Lime Ice par Frozen Freaks ! Ce revigorantmix d'agrumes et de fraicheurravivera le plaisir de vapoter, bouffée après bouffée. Et, grâce à son généreuxflacon 120ml, vous pourrez le booster à votre convenance ! Un mariage acidulé et frais... Nous vous proposons de vous laisser surprendre par une association mystérieuse et puissante de fruits et d'agrumes. Tout cela s'illustre par un mélange depamplemousseet decitron vertpour la note acidulée. Le tout accompagné par une vague de fraicheur intense pour sublimer la recette. Le Grapefruit Lime Ice est ainsi un e-liquide riche en saveurs et en fraicheur qui vous charmera instantanément ! Une recette puissante que l'on doit au fabricant anglais Frozen Freaks. ...en e-liquide 100ml à booster ! En plus d'être un merveilleux mélange fruité et glacé, le e-liquide Grapefruit Lime Ice par Frozen Freaks propose un maxi for

LMM

In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

azure_openai_api_key = os.getenv('AZURE_OPENAI_API_KEY_4')
azure_openai_endpoint = os.getenv('AZURE_OPENAI_API_ENDPOINT_4')
deployment_name = os.getenv('AZURE_DEPLOYMENT_NAME_4')


# Set up Azure OpenAI API (for Azure deployment)
openai.api_type = "azure"
openai.api_key = azure_openai_api_key
openai.api_base = azure_openai_endpoint
openai.api_version = "2023-05-15"  # Ensure this is up to date

import warnings
warnings.filterwarnings('ignore')

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import CSVLoader
from langchain_community.vectorstores import FAISS

In [4]:
llm = AzureChatOpenAI(
    api_key=azure_openai_api_key,  # Azure OpenAI API Key
    api_version="2023-12-01-preview",  # API version (you can adjust it as needed)
    azure_endpoint=azure_openai_endpoint,  # Azure OpenAI endpoint
    deployment_name=deployment_name,  # Ensure deployment_name is the correct one you configured in Azure
    temperature=0  # Optional: Adjust the temperature for randomness in responses
)

C:\Users\antoa\AppData\Local\Temp\ipykernel_32964\1637258887.py:1: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(


In [5]:




# Configuration des embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Chargement des documents
try:
    loader = CSVLoader(file_path='rag_csv.csv', encoding='utf-8')
    documents = loader.load()
    print(f"Nombre de documents chargés : {len(documents)}")
except Exception as e:
    print(f"Erreur lors du chargement des documents : {e}")
    documents = []

# Création et sauvegarde du vecteur
try:
    vectorstore = FAISS.from_documents(documents, embeddings)
    vectorstore.save_local('faiss_vector_store')
except Exception as e:
    print(f"Erreur lors de la création du vectorstore : {e}")

# Chargement du vecteur
vectorstore = FAISS.load_local('faiss_vector_store', embeddings=embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.1}
)

# Définition du prompt
prompt_template = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.

Question: {question}

Context: {context}

Answer:
"""
prompt = ChatPromptTemplate.from_template(prompt_template)

# Pipeline de question-réponse
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

# Test de la chaîne
try:
    question ='Propose moi un produit avec lime'
    response = qa_chain.invoke(question)
    
    print(response)
    
except Exception as e:
    print(f"Erreur lors de l'invocation : {e}")

C:\Users\antoa\AppData\Local\Temp\ipykernel_32964\1118339573.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")



Nombre de documents chargés : 100
Je te propose le produit "Frozen Lemon And Lime 100ml Ice Chuffed". C'est un e-liquide aux saveurs fraîches de citrons jaunes et verts, avec une contenance de 100ml. Il est fabriqué par la marque Chuffed, originaire du Royaume-Uni. Ce produit est surboosté en arômes et peut être ajusté avec des boosters de nicotine selon tes besoins. Voici le lien pour plus de détails : [Frozen Lemon And Lime 100ml Ice Chuffed](https://www.aromes-et-liquides.fr/e-liquide-chuffed/14228-frozen-lemon-and-lime-100ml-ice.html).


In [50]:

import time
import pandas as pd
import numpy as np
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import faiss

# 🔹 Liste des descriptions des produits
descriptions = df['description'].tolist()

# 🔹 Prétraitement des descriptions (tokenization) pour BM25
preprocessed_descriptions = [desc.lower().split() for desc in descriptions]

# 🔹 Initialisation du modèle BM25
bm25 = BM25Okapi(preprocessed_descriptions)

# 🔹 Initialisation du modèle TF-IDF
vectorizer = TfidfVectorizer(stop_words=french_stopwords)
tfidf_matrix = vectorizer.fit_transform(descriptions)

# 🔹 FAISS - Création du vecteur FAISS
embedding_dim = tfidf_matrix.shape[1]  # Dimension des vecteurs TF-IDF
faiss_index = faiss.IndexFlatL2(embedding_dim)  # Index FAISS pour la distance L2

# Ajouter les vecteurs TF-IDF à l'index FAISS
faiss_index.add(tfidf_matrix.toarray().astype(np.float32))

# 🔹 Fonction pour recommander des produits avec FAISS
def recommend_by_faiss(query, faiss_index, vectorizer, df, top_n=5):
    query_tfidf = vectorizer.transform([query]).toarray().astype(np.float32)  # Transformation de la requête en vecteur
    distances, indices = faiss_index.search(query_tfidf, top_n)  # Recherche des produits les plus proches
    
    # 🔸 Création des résultats
    results = [
        {"Nom Produit": df.iloc[idx]['nom_produit'], 
         "Saveur": df.iloc[idx]['saveur'], 
         "Distance": round(distances[0][i], 4)}  # Distance entre les vecteurs
        for i, idx in enumerate(indices[0])
    ]
    
    return results#  Fonction pour recommander des produits avec BM25
def recommend_by_bm25(query, bm25, preprocessed_descriptions, df, top_n=5):
    preprocessed_query = query.lower().split()  # Tokenisation de la requête
    bm25_scores = bm25.get_scores(preprocessed_query)  # Scores BM25
    
    # 🔸 Récupération des indices des produits les plus pertinents
    similar_indices = np.argsort(bm25_scores)[-top_n:][::-1]

    # 🔹 Création des résultats
    results = [
        {"Nom Produit": df.iloc[idx]['nom_produit'], 
         "Saveur": df.iloc[idx]['saveur'], 
         "Score": round(bm25_scores[idx], 4)}
        for idx in similar_indices
    ]

    return results

def recommend_by_tfidf(query, tfidf_matrix, vectorizer, df, top_n=5):
    query_tfidf = vectorizer.transform([query])  # Transformation de la requête
    print(f"Requête transformée en TF-IDF : {query_tfidf.toarray()}")
    
    cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix).flatten()
    print(f"Similarités cosinus : {cosine_similarities}")  # Affiche les similarités cosinus
    
    # Récupérer les indices des produits les plus pertinents
    similar_indices = np.argsort(cosine_similarities)[-top_n:][::-1]

    # Création des résultats
    results = [
        {"Nom Produit": df.iloc[idx]['nom_produit'], 
         "Saveur": df.iloc[idx]['saveur'], 
         "Score": round(cosine_similarities[idx], 4)}
        for idx in similar_indices
    ]
    
    print(f"Résultats recommandés : {results}")  # Vérifie les résultats
    return results

In [54]:
import time
import pandas as pd
import numpy as np

# ⚡ Fonction pour mesurer le temps de réponse
def measure_latency(model_name, func, *args):
    start_time = time.time()
    response = func(*args)  # Exécuter la fonction du modèle
    end_time = time.time()
    
    latency = end_time - start_time
    return {"Modèle": model_name, "Temps (s)": round(latency, 4), "Réponse": response}

# ⚡ Fonction pour récupérer la réponse de BM25
def get_bm25_response(query, bm25, preprocessed_descriptions, df):
    return recommend_by_bm25(query, bm25, preprocessed_descriptions, df)

# ⚡ Fonction pour récupérer la réponse de TF-IDF
def get_tfidf_response(query, tfidf_matrix, vectorizer, df):
    return recommend_by_tfidf(query, tfidf_matrix, vectorizer, df)

# ⚡ Fonction pour récupérer la réponse de Azure OpenAI
def get_llm_response(query):
    return qa_chain.invoke(query)

# 📌 Test avec une requête
query = "Propose-moi un produit avec lime"

# ⚡ Mesurer les performances
results = [
    measure_latency("TF-IDF", get_tfidf_response, query, tfidf_matrix, vectorizer, df),
    measure_latency("BM25", get_bm25_response, query, bm25, preprocessed_descriptions, df),
    measure_latency("FAISS", recommend_by_faiss, query, faiss_index, vectorizer, df),
    measure_latency("Azure OpenAI", get_llm_response, query)
]

# 📊 Créer un DataFrame pour afficher les résultats
df_comparaison = pd.DataFrame(results)




Requête transformée en TF-IDF : [[0. 0. 0. ... 0. 0. 0.]]
Similarités cosinus : [0.00304647 0.08766949 0.         ... 0.04472108 0.00598506 0.        ]
Résultats recommandés : [{'Nom Produit': 'Bloody Lime Fruizee', 'Saveur': 'citron vert, lime, fruits rouges, citron', 'Score': 0.3877}, {'Nom Produit': 'Bloody Lime 50ml Fruizee', 'Saveur': 'baies, lime, citron, baie, citron vert, baies rouges', 'Score': 0.3263}, {'Nom Produit': 'Grapefruit Lime Ice 100ml Frozen Freaks', 'Saveur': 'lime, citron, pamplemousse, citron vert, agrumes', 'Score': 0.2561}, {'Nom Produit': 'Frozen Lemon And Lime 100ml Ice Chuffed', 'Saveur': 'lime, citron, lemon', 'Score': 0.2231}, {'Nom Produit': 'Lime Berry 100ml Paradise Juice 66', 'Saveur': 'baies, lime, citron, baie, citron vert, baies rouges', 'Score': 0.2183}]


In [55]:
df_comparaison

,Modèle,Temps (s),Réponse
0,TF-IDF,0.0120,"[{'Nom Produit': 'Bloody Lime Fruizee', 'Saveu..."
1,BM25,0.0077,[{'Nom Produit': 'Grapefruit Lime Ice 100ml Fr...
2,FAISS,0.0045,"[{'Nom Produit': 'Bloody Lime Fruizee', 'Saveu..."
3,Azure OpenAI,6.3890,Voici quelques produits contenant de la lime :...


In [219]:
# Exemple de DataFrame pour les descriptions de produits
descriptions = df['description'].tolist()

# Initialisation du vectoriseur TF-IDF avec les mots vides en français
vectorizer = TfidfVectorizer(stop_words=french_stopwords)
tfidf_matrix = vectorizer.fit_transform(descriptions)

# Initialiser le modèle BM25 avec les descriptions prétraitées
bm25 = BM25Okapi(preprocessed_descriptions)

# Exemple de requête utilisateur pour BM25
query = "Saveur avec citron"
preprocessed_query = preprocess(query)
scores_bm25 = bm25.get_scores(preprocessed_query)

# Configuration des embeddings pour LLM
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [220]:
# Chargement des documents à partir d'un fichier CSV
try:
    loader = CSVLoader(file_path='rag_csv.csv', encoding='utf-8')
    documents = loader.load()
    print(f"Nombre de documents chargés : {len(documents)}")
except Exception as e:
    print(f"Erreur lors du chargement des documents : {e}")
    documents = []

# Création et sauvegarde du vecteur FAISS
try:
    vectorstore = FAISS.from_documents(documents, embeddings)
    vectorstore.save_local('faiss_vector_store')
except Exception as e:
    print(f"Erreur lors de la création du vectorstore FAISS : {e}")

# Chargement du vecteur FAISS
vectorstore = FAISS.load_local('faiss_vector_store', embeddings=embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.1}
)


Nombre de documents chargés : 100


In [221]:
# Résultats pour chaque modèle
results = []


In [192]:
# 1. Résultat pour TF-IDF
try:
    # Calcul des scores TF-IDF pour la requête
    tfidf_scores = tfidf_matrix * vectorizer.transform([query]).T
    
    # Récupérer le score de similarité maximal
    tfidf_similarity_score = tfidf_scores.max() if tfidf_scores.max() else None

    # Récupérer le document avec le score de similarité maximal
    most_similar_doc_index = tfidf_scores.argmax() if tfidf_scores.max() else None
    tfidf_response = documents[most_similar_doc_index] if most_similar_doc_index is not None else "Aucune réponse trouvée"

    # Ajouter le résultat à la liste des résultats
    results.append({
        'Modele': 'TF-IDF',
        'Similarity': tfidf_similarity_score,
        'Reponse': tfidf_response
    })

except Exception as e:
    print(f"Erreur TF-IDF : {e}")
    results.append({
        'Modele': 'TF-IDF',
        'Similarity': None,
        'Reponse': "Erreur lors de la récupération de la réponse TF-IDF"
    })


In [202]:
result = []

In [222]:
question = "saveur citron"
# 1. Résultats pour TF-IDF
query_tfidf = vectorizer.transform([question])  # Transformer la requête en vecteur TF-IDF
cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix).flatten()  # Calcul de la similarité cosinus

# Sélectionner le produit le plus similaire (le plus grand score de similarité)
best_match_idx = cosine_similarities.argmax()  # Index du produit le plus similaire
tfidf_similarity_score = cosine_similarities[best_match_idx]

results.append({
    'Modele': 'TF-IDF',
    'Similarity': tfidf_similarity_score,
    'Reponse': df['description'][best_match_idx]
})

In [223]:
# Prétraiter les descriptions et la requête
preprocessed_descriptions = [preprocess(desc) for desc in df['description']]

# Initialiser le modèle BM25
bm25 = BM25Okapi(preprocessed_descriptions)

# Exemple de requête utilisateur
question = "saveur citron"

# Prétraiter la requête
preprocessed_query = preprocess(question)

# Calculer la similarité BM25 pour chaque description
scores = bm25.get_scores(preprocessed_query)

# Sélectionner le produit le plus similaire (le plus grand score de similarité)
best_match_idx = scores.argmax()  # Index du produit le plus similaire
bm25_similarity_score = scores[best_match_idx]

# Ajouter le résultat dans la liste 'results'
results.append({
    'Modele': 'BM25',
    'Similarity': bm25_similarity_score,
    'Reponse': df['description'][best_match_idx]
})


In [224]:
# Test de la chaîne
try:
    question = 'Propose moi un produit avec lime'
    
    # Utilisation de similarity_search directement sur le vectorstore pour récupérer les documents similaires
    docs = vectorstore.similarity_search(question, k=5)  # k = 5 documents à récupérer
    
    # Vérification de la structure des documents pour accéder au score
    if docs:
        # Récupérer le score de similarité en accédant au score de chaque document
        faiss_similarity_score = docs[0].score if hasattr(docs[0], 'score') else None  # Vérifier la présence du score
    else:
        faiss_similarity_score = None
    
    # Générer la réponse avec le QA chain
    response = qa_chain.invoke(question)
    
    # Afficher les résultats
    print("Reponse LLM : ", response)
    print("Similarité FAISS : ", faiss_similarity_score)

except Exception as e:
    print(f"Erreur lors de l'invocation : {e}")


Reponse LLM :  Je te propose le produit "Frozen Lemon And Lime 100ml Ice Chuffed". C'est un e-liquide aux saveurs fraîches de citrons jaunes et verts, avec une contenance de 100ml. Il est fabriqué par la marque Chuffed, originaire du Royaume-Uni. Ce produit est surboosté en arômes et peut être ajusté avec des boosters de nicotine selon tes besoins. Voici le lien pour plus de détails : [Frozen Lemon And Lime 100ml Ice Chuffed](https://www.aromes-et-liquides.fr/e-liquide-chuffed/14228-frozen-lemon-and-lime-100ml-ice.html).
Similarité FAISS :  None


In [225]:
import numpy as np

# 4. Résultat pour LLM
try:
    # Obtenir la réponse via qa_chain
    llm_response = qa_chain.invoke(question)
    
    # Calcul de la similarité entre la question et la réponse
    query_embedding = embeddings.embed_query(question)  # Embedding de la question
    llm_embedding = embeddings.embed_query(llm_response)  # Embedding de la réponse
    
    # Vérifier si les embeddings sont sous forme de liste de vecteurs, et extraire le premier vecteur si nécessaire
    query_embedding = np.array(query_embedding[0]) if isinstance(query_embedding, list) else np.array(query_embedding)
    llm_embedding = np.array(llm_embedding[0]) if isinstance(llm_embedding, list) else np.array(llm_embedding)
    
    # Calcul de la similarité cosinus
    llm_similarity_score = np.dot(query_embedding, llm_embedding)  # Utiliser numpy pour le produit scalaire

    # Ajouter les résultats à la liste
    results.append({
        'Modele': 'LLM',
        'Similarity': llm_similarity_score,
        'Reponse': llm_response
    })
except Exception as e:
    print(f"Erreur LLM : {e}")
    results.append({
        'Modele': 'LLM',
        'Similarity': None,
        'Reponse': "Erreur lors de la récupération de la réponse LLM"
    })


In [226]:
# Créer un DataFrame avec les résultats
results_df = pd.DataFrame(results)

# Afficher la table
results_df

,Modele,Similarity,Reponse
0,TF-IDF,0.886548,liquide au citron
1,BM25,0.813998,liquide au citron
2,LLM,0.010548,"Je te propose le produit ""Frozen Lemon And Lime 100ml Ice Chuffed"". C'est un e-liquide aux saveurs fraîches de citrons jaunes et verts, avec une contenance de 100ml. Il est fabriqué par la marque Chuffed, originaire du Royaume-Uni. Ce produit est surboosté en arômes et peut être ajusté avec des boosters de nicotine selon tes besoins. Voici le lien pour plus de détails : [Frozen Lemon And Lime 100ml Ice Chuffed](https://www.aromes-et-liquides.fr/e-liquide-chuffed/14228-frozen-lemon-and-lime-100ml-ice.html)."


In [152]:
import pandas as pd

# Afficher toutes les colonnes sans troncature
pd.set_option('display.max_colwidth', None)

# Afficher toutes les réponses de la colonne 'Reponse'
all_responses = results_df['Reponse']

# Afficher les résultats
print(all_responses)


0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                Aucune 

In [184]:
query_tfidf = vectorizer.transform([query])
print(query_tfidf.shape)  # Vérifier la taille du vecteur de la requête


(1, 5)


In [185]:
print(tfidf_matrix.shape)  # Vérifier la forme de la matrice TF-IDF
if tfidf_matrix.shape[0] == 0:
    print("La matrice TF-IDF est vide")


(4, 5)


In [9]:
# Récupération des documents avec leurs scores
question = "Propose moi un produit avec lime"
retrieved_docs_with_scores = vectorstore.similarity_search_with_score(question, k=5)  # k = nombre max de documents à récupérer

# Affichage des résultats
for doc, score in retrieved_docs_with_scores:
    print(f"Score de similarité: {score:.4f} - Contenu: {doc.page_content[:100]}")



Score de similarité: 0.9838 - Contenu: index: 28
url: https://www.aromes-et-liquides.fr/e-liquide-chuffed/14228-frozen-lemon-and-lime-100ml
Score de similarité: 1.0127 - Contenu: index: 75
url: https://www.aromes-et-liquides.fr/e-liquide-pulp/4838-e-liquide-cherry-frost-par-pulp
Score de similarité: 1.0190 - Contenu: index: 4
url: https://www.aromes-et-liquides.fr/e-liquide-t-juice/3539-e-liquide-red-astaire-par-t-j
Score de similarité: 1.0370 - Contenu: index: 45
url: https://www.aromes-et-liquides.fr/e-liquide-swoke/1722-e-liquide-bisou-v2-par-swoke.h
Score de similarité: 1.0371 - Contenu: index: 23
url: https://www.aromes-et-liquides.fr/e-liquide-eliquid-france/2525-exotic-eliquid-france


In [8]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Encoder la question en embedding
query_embedding = embeddings.embed_query(question)

# Récupérer les documents et leurs embeddings
retrieved_docs = retriever.get_relevant_documents(question)
doc_embeddings = [embeddings.embed_query(doc.page_content) for doc in retrieved_docs]

# Calculer les similarités cosinus
similarities = cosine_similarity([query_embedding], doc_embeddings)[0]

# Afficher les résultats
for doc, score in zip(retrieved_docs, similarities):
    print(f"Score Cosine Similarity: {score:.4f} - Contenu: {doc.page_content[:200]}")


Score Cosine Similarity: 0.4372 - Contenu: index: 8
url: https://www.aromes-et-liquides.fr/e-liquide-pulp/1832-e-liquide-fraise-sauvage-pulp.html
nom_produit: Fraise Sauvage PULP
img_produit: https://assets.aromes-et-liquides.fr/54914-thickbox
Score Cosine Similarity: 0.4311 - Contenu: index: 16
url: https://www.aromes-et-liquides.fr/e-liquide-alfaliquid/2307-e-liquide-miss-fraise-par-alfaliquid.html
nom_produit: Miss Fraise Alfaliquid
img_produit: https://assets.aromes-et-liquides.
Score Cosine Similarity: 0.4231 - Contenu: index: 99
url: https://www.aromes-et-liquides.fr/e-liquide-cirkus-vdlv/10230-e-liquide-cirkus-fraise-menthe-par-vdlv.html
nom_produit: Fraise Menthe Cirkus VDLV
img_produit: https://assets.aromes-et-l
Score Cosine Similarity: 0.4189 - Contenu: index: 81
url: https://www.aromes-et-liquides.fr/e-liquide-fruizee/3886-e-liquide-fruizee-cassis-mangue-par-eliquid-france.html
nom_produit: Cassis Mangue Fruizee
img_produit: https://assets.aromes-et


In [10]:
df = pd.read_csv('rag_csv.csv')

In [12]:
# TF-IDF Similarity
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['description'])

def recommend_tfidf(query, tfidf_matrix, vectorizer):
    query_tfidf = vectorizer.transform([query])
    cosine_sim = cosine_similarity(query_tfidf, tfidf_matrix).flatten()
    idx = cosine_sim.argsort()[-1]  # Meilleur match
    return df['description'][idx], cosine_sim[idx]


In [14]:
from rank_bm25 import BM25Okapi

In [15]:
#  BM25 Similarity

preprocessed_descriptions = [desc.lower().split() for desc in df['description']]
bm25 = BM25Okapi(preprocessed_descriptions)

def recommend_bm25(query):
    preprocessed_query = query.lower().split()
    scores = bm25.get_scores(preprocessed_query)
    idx = scores.argsort()[-1]  # Meilleur match
    return df['description'][idx], scores[idx]

In [16]:
# LLM + FAISS (Vector Search)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_texts(df['description'].tolist(), embeddings)

def recommend_llm(query):
    retrieved_docs = vectorstore.similarity_search_with_score(query, k=1)  # k=1 pour le meilleur match
    if retrieved_docs:
        return retrieved_docs[0][0].page_content, retrieved_docs[0][1]
    return "Aucun résultat", 0.0  # Si aucun document trouvé


In [17]:
# Comparaison des Modèles

query = "Saveur cassis lime"

# Obtenir les résultats des trois modèles
tfidf_result, tfidf_score = recommend_tfidf(query, tfidf_matrix, vectorizer)
bm25_result, bm25_score = recommend_bm25(query)
llm_result, llm_score = recommend_llm(query)


In [18]:
# Création du tableau de comparaison
comparison_df = pd.DataFrame({
    "Modèle": ["TF-IDF", "BM25", "LLM (FAISS + OpenAI)"],
    "Similarité": [tfidf_score, bm25_score, llm_score],
    "Réponse": [tfidf_result, bm25_result, llm_result]
})

In [20]:
comparison_df

,Modèle,Similarité,Réponse
0,TF-IDF,0.114903,Le Frozen Lemon And Lime 100ml est un e-liquid...
1,BM25,6.130871,Le Frozen Lemon And Lime 100ml est un e-liquid...
2,LLM (FAISS + OpenAI),1.271297,Le Cassis Mangue est un e-liquide français pou...


normalisation des score

In [21]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Supposons que ces scores proviennent de tes modèles
scores = {
    "TF-IDF": tfidf_score,
    "BM25": bm25_score,
    "LLM (FAISS + OpenAI)": llm_score
}

# Convertir les scores en dataframe
df_scores = pd.DataFrame(list(scores.items()), columns=["Modèle", "Score"])

# Initialiser le scaler MinMax
scaler = MinMaxScaler()

# Normaliser les scores (de 0 à 1)
df_scores["Score Normalisé"] = scaler.fit_transform(df_scores[["Score"]])

In [22]:
df_scores

,Modèle,Score,Score Normalisé
0,TF-IDF,0.114903,0.000000
1,BM25,6.130871,1.000000
2,LLM (FAISS + OpenAI),1.271297,0.192221


In [23]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Supposons que ces scores proviennent de tes modèles
scores = {
    "TF-IDF": tfidf_score,
    "BM25": bm25_score,
    "LLM (FAISS + OpenAI)": llm_score
}

# Convertir les scores en dataframe
df_scores = pd.DataFrame(list(scores.items()), columns=["Modèle", "Score"])

# Initialiser le scaler Z-score
scaler = StandardScaler()

# Normaliser les scores (moyenne 0, écart-type 1)
df_scores["Score Normalisé"] = scaler.fit_transform(df_scores[["Score"]])

# Afficher le résultat
print(df_scores)


                 Modèle     Score  Score Normalisé
0                TF-IDF  0.114903        -0.917241
1                  BM25  6.130871         1.390823
2  LLM (FAISS + OpenAI)  1.271297        -0.473583


Évaluation basée sur des règles, où nous vérifions si la recommandation contient des mots-clés présents dans la requête.
Évaluation avec une liste d'évaluateurs pour recueillir des évaluations manuelles des recommandations.

In [25]:
# 1. Évaluation basée sur des règles (Automatique)
def is_recommendation_correct(query, recommendation):
    """
    Vérifie si la recommandation contient des mots-clés présents dans la requête.
    """
    query_keywords = set(query.lower().split())  # Divise la requête en mots-clés
    recommendation_keywords = set(recommendation.lower().split())  # Divise la recommandation en mots-clés
    
    # Vérifie si les mots-clés de la requête sont dans la recommandation
    return len(query_keywords & recommendation_keywords) > 0

In [26]:
# 2. Liste d'évaluations manuelles
evaluations = {
    "TF-IDF": [
        {"query": "Saveur cassis lime", "recommandation": "Saveur cassis lime", "eval": 1},  # 1 = correcte
        {"query": "Saveur pomme fraîche", "recommandation": "Saveur pomme cassis", "eval": 0}, # 0 = incorrecte
        {"query": "Cassis et lime saveur", "recommandation": "Saveur citron cassis", "eval": 1}
    ],
    "BM25": [
        {"query": "Saveur cassis lime", "recommandation": "Saveur cassis lime", "eval": 1},
        {"query": "Saveur pomme fraîche", "recommandation": "Saveur pomme fraîche", "eval": 1},
        {"query": "Cassis et lime saveur", "recommandation": "Saveur lime cassis", "eval": 1}
    ],
    "LLM (FAISS + OpenAI)": [
        {"query": "Saveur cassis lime", "recommandation": "Saveur cassis lime", "eval": 1},
        {"query": "Saveur pomme fraîche", "recommandation": "Saveur pomme fraîche", "eval": 1},
        {"query": "Cassis et lime saveur", "recommandation": "Saveur cassis lime", "eval": 1}
    ]
}

In [27]:
# Calcul de la précision à partir des évaluations manuelles
precision_scores_manual = {
    model: sum([eval_info['eval'] for eval_info in evaluations[model]]) / len(evaluations[model])
    for model in evaluations
}

In [28]:
# 3. Calcul de la précision à partir de l'évaluation basée sur des règles
correct_recommendations = {
    "TF-IDF": [
        is_recommendation_correct("Saveur cassis lime", "Saveur cassis lime"),
        is_recommendation_correct("Saveur pomme fraîche", "Saveur pomme cassis"),
        is_recommendation_correct("Cassis et lime saveur", "Saveur citron cassis")
    ],
    
    "BM25": [
        is_recommendation_correct("Saveur cassis lime", "Saveur cassis lime"),
        is_recommendation_correct("Saveur pomme fraîche", "Saveur pomme fraîche"),
        is_recommendation_correct("Cassis et lime saveur", "Saveur lime cassis")
    ],
    
    "LLM (FAISS + OpenAI)": [
        is_recommendation_correct("Saveur cassis lime", "Saveur cassis lime"),
        is_recommendation_correct("Saveur pomme fraîche", "Saveur pomme fraîche"),
        is_recommendation_correct("Cassis et lime saveur", "Saveur cassis lime")
    ]
}

In [29]:
# Calcul de la précision à partir des règles
precision_scores_rule_based = {
    model: sum(correct_recommendations[model]) / len(correct_recommendations[model])
    for model in correct_recommendations
}

In [30]:
# Afficher les résultats
print("Précision des modèles (évaluation manuelle) :", precision_scores_manual)
print("Précision des modèles (basée sur des règles) :", precision_scores_rule_based)

Précision des modèles (évaluation manuelle) : {'TF-IDF': 0.6666666666666666, 'BM25': 1.0, 'LLM (FAISS + OpenAI)': 1.0}
Précision des modèles (basée sur des règles) : {'TF-IDF': 1.0, 'BM25': 1.0, 'LLM (FAISS + OpenAI)': 1.0}


In [31]:
# Comparer les résultats entre l'évaluation manuelle et l'évaluation basée sur des règles
for model in evaluations:
    print(f"\nModèle: {model}")
    print(f"Précision (évaluation manuelle) : {precision_scores_manual[model]}")
    print(f"Précision (évaluation basée sur des règles) : {precision_scores_rule_based[model]}")


Modèle: TF-IDF
Précision (évaluation manuelle) : 0.6666666666666666
Précision (évaluation basée sur des règles) : 1.0

Modèle: BM25
Précision (évaluation manuelle) : 1.0
Précision (évaluation basée sur des règles) : 1.0

Modèle: LLM (FAISS + OpenAI)
Précision (évaluation manuelle) : 1.0
Précision (évaluation basée sur des règles) : 1.0


In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Liste de descriptions (plusieurs descriptions de produits)
descriptions = [
    "Le e-liquide Ragnarok par A&L Ultimate est un produit prêt à l'emploi pour cigarette électronique, avec des saveurs de fruits rouges et de cassis.",
    "Ce e-liquide fruité à la fraise est parfait pour ceux qui aiment une touche sucrée de fruits frais.",
    "Le e-liquide tropical mélange des saveurs de mangue, d'ananas et de fruits de la passion."
]

# Requête utilisateur
requete_utilisateur = "e-liquide fruité à la fraise"

# Initialiser un TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Convertir les descriptions et la requête en vecteurs TF-IDF
tfidf_matrix = vectorizer.fit_transform(descriptions + [requete_utilisateur])

# Calculer la similarité cosinus entre la requête et toutes les descriptions
similarity_scores = np.dot(tfidf_matrix[-1].toarray(), tfidf_matrix[:-1].toarray().T)

# Récupérer le meilleur score (le score maximal)
best_similarity_score = np.max(similarity_scores)

# Afficher le meilleur score
print(f"Le meilleur score de similarité cosinus entre la requête et les descriptions est : {best_similarity_score}")


Le meilleur score de similarité cosinus entre la requête et les descriptions est : 0.3848816585632686


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np


# Requête utilisateur
requete_utilisateur = "e-liquide fruité à la fraise"

# Initialiser un TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Convertir les descriptions du DataFrame et la requête en vecteurs TF-IDF
tfidf_matrix = vectorizer.fit_transform(df['description'].values.tolist() + [requete_utilisateur])

# Calculer la similarité cosinus entre la requête et toutes les descriptions
similarity_scores = np.dot(tfidf_matrix[-1].toarray(), tfidf_matrix[:-1].toarray().T)

# Récupérer le meilleur score (le score maximal)
best_similarity_score = np.max(similarity_scores)

# Afficher le meilleur score
print(f"Le meilleur score de similarité cosinus entre la requête et les descriptions est : {best_similarity_score}")


Le meilleur score de similarité cosinus entre la requête et les descriptions est : 0.19586681511555729


In [53]:
from rank_bm25 import BM25Okapi

# Liste de descriptions (plusieurs descriptions de produits)
descriptions = [
    "Le e-liquide Ragnarok par A&L Ultimate est un produit prêt à l'emploi pour cigarette électronique, avec des saveurs de fruits rouges et de cassis.",
    "Ce e-liquide fruité à la fraise est parfait pour ceux qui aiment une touche sucrée de fruits frais.",
    "Le e-liquide tropical mélange des saveurs de mangue, d'ananas et de fruits de la passion."
]

# Requête utilisateur
requete_utilisateur = "e-liquide fruité à la fraise"

# Tokenisation des descriptions et de la requête
descriptions_tokens = [description.split() for description in descriptions]
requete_tokens = requete_utilisateur.split()

# Construire un modèle BM25 avec les descriptions
bm25 = BM25Okapi(descriptions_tokens)

# Calculer la similarité BM25 entre la requête et toutes les descriptions
bm25_scores = bm25.get_scores(requete_tokens)

# Récupérer le meilleur score (le score maximal)
best_score = max(bm25_scores)

# Afficher le meilleur score
print(f"Le meilleur score BM25 entre la requête et les descriptions est : {best_score}")


Le meilleur score BM25 entre la requête et les descriptions est : 1.1403394301403824


In [80]:
# Requête utilisateur
requete_utilisateur = "e-liquide fruité à la fraise"

# Tokenisation des descriptions du DataFrame et de la requête
descriptions_tokens = [description.split() for description in df['description'].tolist()]
requete_tokens = requete_utilisateur.split()

# Construire un modèle BM25 avec les descriptions
bm25 = BM25Okapi(descriptions_tokens)

# Calculer la similarité BM25 entre la requête et toutes les descriptions
bm25_scores = bm25.get_scores(requete_tokens)

# Normalisation Min-Max des scores BM25
scaler = MinMaxScaler()
bm25_scores_normalized = scaler.fit_transform(bm25_scores.reshape(-1, 1)).flatten()

# Récupérer le meilleur score normalisé (le score maximal)
best_normalized_score = max(bm25_scores_normalized)

# Afficher le meilleur score normalisé
print(f"Le meilleur score BM25 normalisé entre la requête et les descriptions est : {best_normalized_score}")

Le meilleur score BM25 normalisé entre la requête et les descriptions est : 0.9999999999999998


In [89]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Fonction pour calculer la similarité cosinus entre deux textes
def calculate_similarity(query, context, embeddings):
    # Calculer les embeddings pour la requête et le contexte
    query_embedding = embeddings.embed_query(query)
    context_embedding = embeddings.embed_query(context)

    # Calculer la similarité cosinus entre les deux embeddings
    similarity_score = cosine_similarity([query_embedding], [context_embedding])
    
    return similarity_score[0][0]  # Récupérer la valeur de la similarité

# Modifiez la fonction d'évaluation pour inclure la similarité
def evaluate_similarity_with_score(query):
    # Récupérer le contexte pertinent
    context = get_relevant_context(query)
    
    # Calculer le score de similarité
    similarity_score = calculate_similarity(query, context, embeddings)
    
    # Afficher le score de similarité
    print(f"Score de similarité entre la requête et le contexte : {similarity_score}")
    
    return similarity_score  # Retourner seulement le score de similarité

# Exemple de requête
requete_utilisateur = "e-liquide fruité à la fraise"
similarity_score = evaluate_similarity_with_score(requete_utilisateur)


Score de similarité entre la requête et le contexte : 0.6568676602603319


In [99]:
df.columns

Index(['index', 'url', 'nom_produit', 'img_produit', 'prix_produit',
       'contenance', 'pg_vg', 'origine', 'frais', 'surbooste', 'saveur',
       'description', 'brand', 'gout', 'info_brand', 'id_produit',
       'similarity_score'],
      dtype='object')

In [101]:
# Filtrer les lignes où le titre est "Frozen Lemon And Lime 100ml Ice Chuffed"
resultat  = df[df['nom_produit'] == 'Frozen Lemon And Lime 100ml Ice Chuffed']

In [102]:
# Afficher la colonne 'description' pour les lignes filtrées
if not resultat.empty:
    descriptions = resultat['description'].tolist()
    for desc in descriptions:
        print(desc)
else:
    print("Aucune ligne correspondante trouvée.")

Le Frozen Lemon And Lime 100ml est un e-liquide à booster issu de la collection Ice de la marque Chuffed, auxsaveurs fraîches de citrons jaunes et verts. Sa recette fabriquée en France se compose d'arômes alimentaires, de propylène glycol (PG) et de glycérine végétale (VG), pour un ratio 30PG/70VG. Présenté dans une fiole de 120ml dotée d'un bouchon sécurisé et d'un embout fin,vous pouvez au besoin lui additionnerdu liquide de base ou des boosters 10ml de nicotine (non inclus). Qui est Chuffed ? Chuffed est une marque originaire du Royaume-Uni, spécialisée dans la conception de e-liquides pour cigarette électronique. Retrouvez les recettes de Chuffed en format de 100ml, non nicotinés de base et à booster en fonction de vos besoins. Comment préparer votre e-liquide Frozen Lemon And Lime ? Ce e-liquide est proposé en version100mlboostée en arômes dans un flacon de120ml. Si vous le souhaitez, vous pouvez ajouter un ou plusieurs boosters de nicotine selon vos besoins sans perte de saveur. 